In [1]:
from tsr.system import TSR
pretrained_model_name_or_path = "stabilityai/TripoSR"
model = TSR.from_pretrained(
    pretrained_model_name_or_path,
    config_name="config.yaml",
    weight_name="model.ckpt",
)

/home/shangyu/Cyberpunk8T/anaconda_space/envs/langsplat/lib/python3.9/site-packages/numba/np/ufunc/parallel.py:371: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12050. The TBB threading layer is disabled.
  warnings.warn(problem)


In [2]:
model

TSR(
  (image_tokenizer): DINOSingleImageTokenizer(
    (model): ViTModel(
      (embeddings): ViTEmbeddings(
        (patch_embeddings): ViTPatchEmbeddings(
          (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
        )
        (dropout): Dropout(p=0.0, inplace=False)
      )
      (encoder): ViTEncoder(
        (layer): ModuleList(
          (0-11): 12 x ViTLayer(
            (attention): ViTAttention(
              (attention): ViTSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.0, inplace=False)
              )
              (output): ViTSelfOutput(
                (dense): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.0, inplace=False)
              )
            )
            (

In [1]:
import argparse
import logging
import os
import time

import numpy as np
import rembg
import torch
from PIL import Image

from tsr.system import TSR
from tsr.utils import remove_background, resize_foreground, save_video
no_remove_bg = False
image = ['examples/unicorn.png']
images = []
foreground_ratio = 0.85
if no_remove_bg:
    rembg_session = None
else:
    rembg_session = rembg.new_session()
for i, image_path in enumerate(image):
    if no_remove_bg:
        image = np.array(Image.open(image_path).convert("RGB"))
    else:
        image = remove_background(Image.open(image_path), rembg_session)
        image = resize_foreground(image, foreground_ratio)
        image = np.array(image).astype(np.float32) / 255.0
        image = image[:, :, :3] * image[:, :, 3:4] + (1 - image[:, :, 3:4]) * 0.5
        image = Image.fromarray((image * 255.0).astype(np.uint8))
        # if not os.path.exists(os.path.join(output_dir, str(i))):
        #     os.makedirs(os.path.join(output_dir, str(i)))
        # image.save(os.path.join(output_dir, str(i), f"input.png"))
    images.append(image)

/home/shangyu/Cyberpunk8T/anaconda_space/envs/langsplat/lib/python3.9/site-packages/numba/np/ufunc/parallel.py:371: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12050. The TBB threading layer is disabled.
  warnings.warn(problem)
2024-03-06 20:54:23.133134132 [W:onnxruntime:Default, onnxruntime_pybind_state.cc:614 CreateExecutionProviderInstance] Failed to create TensorrtExecutionProvider. Please reference https://onnxruntime.ai/docs/execution-providers/TensorRT-ExecutionProvider.html#requirements to ensure all dependencies are met.


In [4]:
device = 'cpu'
with torch.no_grad():
        scene_codes = model([image, image], device=device)

In [5]:
scene_codes.shape

torch.Size([2, 3, 40, 64, 64])

In [6]:
b, n, c, h, w = scene_codes.shape
scene_tokens = scene_codes.permute(0, 2, 1, 3, 4).reshape(b, n*c, h, w)
scene_tokens.shape

torch.Size([2, 120, 64, 64])

In [2]:
from triplane_clip import TriPlaneCLIPVisionModel, TriPlaneCLIPVisionConfig
from transformers import CLIPVisionModel, CLIPProcessor
vision_config = TriPlaneCLIPVisionConfig()
vision = TriPlaneCLIPVisionModel(vision_config)
clip_vision = CLIPVisionModel.from_pretrained("openai/clip-vit-large-patch14")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-large-patch14")

In [3]:
import torch
out = vision(images = images)
# out = vision(pixel_values = scene_tokens)

In [4]:
out.pooler_output.shape

torch.Size([1, 1024])

In [12]:
inputs = processor(images=image, return_tensors="pt")
clip_out = clip_vision(**inputs)

In [14]:
clip_out.pooler_output.shape

torch.Size([1, 1024])

In [16]:
from torch import nn

nn.MSELoss()(out.pooler_output, clip_out.pooler_output)

tensor(2.0791, grad_fn=<MseLossBackward0>)

In [1]:
import os

img = os.listdir('data/objaverse/Cap3D_imgs_view0')

In [2]:
img[0]

'8b5e46f712e9450cab09bf31fb29e3c5'

In [4]:
from PIL import Image
image = Image.open('data/objaverse/Cap3D_imgs_view0/8b5e46f712e9450cab09bf31fb29e3c5')

In [5]:
image.size

(512, 512)